<h1><center> AI&ML Experts on Twitter

<img src="plan.PNG" style="width: 800px">

<h4>Setup

In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd

<h2> 1 Data Preparation

In [2]:
def getNewUsers(oldUsers,userMentions):
    oldUsers=[i.lower() for i in oldUsers]
    userMentions=[i.lower() for i in userMentions]
    NewUsers=[userID for userID in userMentions if userID not in oldUsers]
    return NewUsers

In [3]:
def getTweetSource(tweetSource):
    #tweetSource.unique()
    return data.tweetSource.describe().top

In [4]:
def getHashtags(hashtags):
    result={}
    
    for hashtag in hashtags:
        if type(hashtag) ==str:
            hashtag_split=hashtag[1:-1].replace("'",'"').replace('}, {','}}, {{').split('}, {')   
            for hash in hashtag_split:
                #print(hash)
                t=json.loads(hash)
                if t['text'] in result:
                    result[t['text']]+=1
                else:
                    result[t['text']]=1
    return result

In [5]:
def getUserMentions(hashtags):
    import re
    
    result={}
    
    for hashtag in hashtags:
        if type(hashtag) ==str:
            hashtag=re.sub("[a-zA-Z0-9]\'[a-zA-Z0-9]",' ',hashtag)
            hashtag=re.sub("[a-zA-Z0-9]\' ",' ',hashtag)
            #hashtag=re.sub('["*'*"]',' ',hashtag)
            
            '''t=re.finditer('\"*\"',hashtag)
            if t != None:
                print(i.span() for i in t)'''
            
            #hashtag=re.sub('.\\.',' ',hashtag)
            #hashtag=hashtag.replace('\\','')
            #hashtag=hashtag.replace("'yoav)",'yoav)')
            hashtag_split=hashtag[1:-1].replace("'",'"').replace('}, {','}}, {{').split('}, {')  
            for hash in hashtag_split:
                #print(hash)
                hash_split=hash.replace('", "','"", ""').split('", "')
                #print(hash_split[0][1:])
                t=json.loads("{"+hash_split[0][1:]+"}",strict=False)
                if t['screen_name'] in result:
                    result[t['screen_name']]+=1
                else:
                    result[t['screen_name']]=1
    return result

<h2>2 Pre-Processing

In [6]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk import word_tokenize

In [7]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 

#----add more stopword and pubc-----#
stop.add('rt')
stop.add('nan')
exclude.add('...')
exclude.add('"')
exclude.add('...')
#---------------------------#

lemma = WordNetLemmatizer()

def clean(doc):
    doc=doc.split(', ')
    doc=" ".join(doc)
    #is_char = " ".join([word for word in doc.split(' ') if word.isalpha()])
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    #punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in stop_free.split())    
    is_word = " ".join([word for word in normalized.split(' ') if len(word)>2])
    
    '''
    print(doc)
    print(is_char)
    print(stop_free)
    print(normalized)
    print(is_word) 
    '''
    
    return is_word

#tokens0w = word_tokenize(text0)

<h2>Modelling

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [9]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
#model = KNeighborsClassifier(n_neighbors=3)
from sklearn.ensemble import RandomForestClassifier 
#model = RandomForestClassifier(n_estimators = 100, n_jobs=-1) # try 1000

from sklearn.ensemble import GradientBoostingClassifier

<h2>Validation and Testing

In [10]:
from sklearn import metrics
from sklearn.cross_validation import train_test_split

def evalResults(y_test, y_pred): 
    a=metrics.accuracy_score(y_test, y_pred)
    b=metrics.confusion_matrix(y_test, y_pred)
    c=metrics.precision_recall_fscore_support(y_test, y_pred)
    '''print("Accuracy score: ",a)
    print("Confusion matrix: ",b)
    print("precision_recall_fscore: ",c)'''
    print(metrics.classification_report(y_test, y_pred))
    print("Accuracy: ",metrics.accuracy_score(y_test,y_pred))
    print(b)
    dictResults= {'Accuracy score': a, 'Confusion matrix': b, 'precision_recall_fscore': c}
    return dictResults

C:\Users\Shuttle\Anaconda2\envs\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


<h3>Feature = Description

In [11]:
df=pd.read_csv('Data_Training_Simple_Full.csv')
description=df.description.fillna('_')
X=[clean(str(i)) for i in description]
y=df.follow

In [38]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split =  train_test_split(X, y, test_size=0.4, random_state =0)

In [39]:
model2 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('best', TruncatedSVD(n_components=50)),
])
model2.fit(x_train,y_train)
#print(model.get_feature_names())
#model2.steps[1].singular_values_ 

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=None, tol=0.0))])

In [40]:
model=TfidfVectorizer(ngram_range=(1,3))
model.fit(x_train,y_train)
#print(model.get_feature_names())
dft=pd.DataFrame(model.idf_ )
dft.describe()

,0
count,3942.000000
mean,5.257869
std,0.244502
min,2.643339
25%,5.317488
50%,5.317488
75%,5.317488
max,5.317488


In [13]:
pipeline_text = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('best', TruncatedSVD(n_components=50)),
    ('model', RandomForestClassifier(n_estimators = 100, n_jobs=-1)),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [14]:
pipeline_text.fit(x_train,y_train)
y_pred = pipeline_text.predict(x_test)
dictResults=evalResults(y_test, y_pred)

             precision    recall  f1-score   support

          0       1.00      0.14      0.24        43
          1       0.61      1.00      0.75        57

avg / total       0.78      0.63      0.54       100

Accuracy:  0.63
[[ 6 37]
 [ 0 57]]


<h3>Feature = TweetSource

In [41]:
df=pd.read_csv('Data_Training_Simple_Full.csv')
df = shuffle(df).reset_index(drop=True)

X=pd.get_dummies(df.tweetSource)
y=df.follow

In [42]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split =  train_test_split(X, y, test_size=0.4, random_state =0)

In [43]:
pipeline_text = Pipeline([
    ("pca", PCA(n_components=5)), 
    ("univ_select", SelectKBest(k=3)),
    ('model', RandomForestClassifier(n_estimators = 100, n_jobs=-1)),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [44]:
pipeline_text.fit(x_train,y_train)
y_pred = pipeline_text.predict(x_test)
dictResults=evalResults(y_test, y_pred)

             precision    recall  f1-score   support

          0       0.56      0.24      0.34        37
          1       0.67      0.89      0.76        63

avg / total       0.63      0.65      0.61       100

Accuracy:  0.65
[[ 9 28]
 [ 7 56]]


<h3>Hetero Features

In [195]:
from sklearn.cluster import KMeans, MiniBatchKMeans
#m = KMeans(n_clusters=2, init='k-means++', max_iter=100, n_init=1)

pipeline = Pipeline([
    # Use FeatureUnion to combine the features from description and tweetSource
    ('union', FeatureUnion(
        transformer_list=[

            ('tweetSource', Pipeline([
                ('selector', ItemSelector(key='tweetSource')),
                ("pca", PCA(n_components=3)), 
                ("univ_select", SelectKBest(k=3)),
            ])),

            # Pipeline for bag-of-words model for description
            ('description', Pipeline([
                ('selector', ItemSelector(key='description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2),use_idf=False)),
                ('svd', TruncatedSVD(n_components=5)),

            ])),
        ],

        # weight components in FeatureUnion
        transformer_weights={
            'tweetSource': 1,
            'description': 1,
        },
    )),

    # Use a SVC classifier on the combined features
    #('tree', RandomForestClassifier(n_estimators = 10, n_jobs=-1)),
    ('keans',KMeans(n_clusters=2, init='k-means++', max_iter=100, n_init=1)),
    #("grad_boost",GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=10)),
])

In [187]:
'''from sklearn.model_selection import GridSearchCV
parameters = {'union__transformer_weights': [{"tweetSource":1,"description":0}, 
                                            {"tweetSource":0,"description":1}, 
                                            {"tweetSource":2,"description":1},
                                            {"tweetSource":1,"description":2},
                                            {"tweetSource":1,"description":1}],
}
clf = GridSearchCV(pipeline, parameters, n_jobs=-1)
clf.fit(x_train,y_train)

print(clf.best_estimator_)
print(clf.best_score_)
print(clf.best_params_)'''

'from sklearn.model_selection import GridSearchCV\nparameters = {\'union__transformer_weights\': [{"tweetSource":1,"description":0}, \n                                            {"tweetSource":0,"description":1}, \n                                            {"tweetSource":2,"description":1},\n                                            {"tweetSource":1,"description":2},\n                                            {"tweetSource":1,"description":1}],\n}\nclf = GridSearchCV(pipeline, parameters, n_jobs=-1)\nclf.fit(x_train,y_train)\n\nprint(clf.best_estimator_)\nprint(clf.best_score_)\nprint(clf.best_params_)'

In [201]:
from sklearn.utils import shuffle

df=pd.read_csv('Data_Training_Simple_Full.csv')
df = shuffle(df).reset_index(drop=True)

n=1
k=int(n*len(df))
print(k)

tweetSource=pd.get_dummies(df.tweetSource)

description=df.description.fillna('_')
description=[clean(str(i)) for i in description]

y_train=df.follow[:k]
y_test=df.follow[k:]
x_train={'tweetSource':tweetSource.iloc[:k,:], 'description':description[:k]}
x_test={'tweetSource':tweetSource.iloc[k:,:], 'description':description[k:]}

pipeline.fit(x_train,y_train)
y_pred = pipeline.predict(x_test)
dictResults=evalResults(y_test, y_pred)

249


Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('tweetSource', Pipeline(memory=None,
     steps=[('selector', ItemSelector(key='tweetSource')), ('pca', PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('univ_sele...=2, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0))])

<h3>Test on new users

In [202]:
df=pd.read_csv('newUsers1.csv')
from sklearn.utils import shuffle

df = shuffle(df).reset_index(drop=True)

tweetSource=pd.get_dummies(df.tweetSource)

description=df.description.fillna('_')
description=[clean(str(i)) for i in description]

print(tweetSource.shape)
print(len(description))

x_test={'tweetSource':tweetSource, 'description':description}
y_pred = pipeline.predict(x_test)
df.insert(0,'pred_follow',y_pred)
df[df.userID != 'test']

(31, 20)
31


,pred_follow,userID,category,follow,description,num_followers,tweets,hashtags,userMentions,userLocation,userTimezone,tweetSource
1,1,rstudio,NaN,NaN,Open source and enterprise-ready professional ...,55476.0,NaN,NaN,NaN,Boston,Eastern Time (US & Canada),Twitter Web Client
2,0,projectjupyter,NaN,NaN,Multi-language interactive computing environme...,24348.0,NaN,NaN,NaN,The Future,NaN,TweetDeck
4,1,pfau,NaN,NaN,I study brains and intelligence. Anarchist bio...,2221.0,NaN,NaN,NaN,London,Eastern Time (US & Canada),Twitter Web Client
5,1,catherineols,NaN,NaN,Research engineer @googlebrain. Past: software...,2905.0,NaN,NaN,NaN,NaN,NaN,Twitter Web Client
8,1,ch402,NaN,NaN,ML Researcher. I want to understand things cle...,23225.0,NaN,NaN,NaN,"San Francisco, CA",NaN,Twitter Web Client
9,0,iamtrask,NaN,NaN,"PhD Student @UniofOxford, Research Scientist @...",20900.0,NaN,NaN,NaN,"Oxford,UK",NaN,Twitter for iPhone
20,1,ilyasut,NaN,NaN,Research @ OpenAI,8440.0,NaN,NaN,NaN,NaN,NaN,Twitter Web Client
21,1,beenwrekt,NaN,NaN,optimization. machine learning. uc berkeley. i...,3318.0,NaN,NaN,NaN,"Berkeley, CA",Pacific Time (US & Canada),Twitter Web Client
22,1,distillpub,NaN,NaN,Supporting Clarity in Machine Learning. https:...,14838.0,NaN,NaN,NaN,NaN,NaN,Twitter Web Client
29,1,timnitgebru,NaN,NaN,NaN,1851.0,NaN,NaN,NaN,NaN,Alaska,Twitter Web Client


In [203]:
df.to_csv('new_user_pred_kmeans.csv',index=0)

<h3>Test code

In [199]:
n=0.7
k=int(n*len(df))
acc=[]

for i in range(100):
    df = shuffle(df).reset_index(drop=True)

    tweetSource=pd.get_dummies(df.tweetSource)

    description=df.description.fillna('_')
    description=[clean(str(i)) for i in description]

    y_train=df.follow[:k]
    y_test=df.follow[k:]
    x_train={'tweetSource':tweetSource.iloc[:k,:], 'description':description[:k]}
    x_test={'tweetSource':tweetSource.iloc[k:,:], 'description':description[k:]}

    pipeline.fit(x_train,y_train)
    y_pred = pipeline.predict(x_test)
    dictResults=evalResults(y_test, y_pred)
    acc.append(dictResults.get('Accuracy score'))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [140]:
pd.DataFrame(acc).describe()

,0
count,100.000000
mean,0.701467
std,0.043418
min,0.560000
25%,0.676667
50%,0.706667
75%,0.733333
max,0.800000


<h2>Ensemble methods

<h3>Description

In [52]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection

num_trees = 10
seed=42
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('best', TruncatedSVD(n_components=50)),
    ('boost',model)
])

df=pd.read_csv('Data_Training_Simple_Full.csv')
description=df.description.fillna('_')
X=[clean(str(i)) for i in description]
y=df.follow

results = model_selection.cross_val_score(pipeline, X, y, cv=kfold)
results

array([0.76, 0.76, 0.88, 0.92, 0.88, 0.56, 0.44, 0.56, 0.8 , 0.5 ])

<h3>Tweet Source

In [35]:
df=pd.read_csv('Data_Training_Simple_Full.csv')
df = shuffle(df).reset_index(drop=True)

X=pd.get_dummies(df.tweetSource)
y=df.follow

model_boost = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model_boost, X, y, cv=kfold)
results

array([0.68      , 0.68      , 0.6       , 0.48      , 0.76      ,
       0.68      , 0.6       , 0.76      , 0.56      , 0.66666667])

<Parameter Tuning>

<h2>Parameter Tuning

In [25]:
df=pd.read_csv('Data_Training_Simple_Full.csv')

tweetSource=pd.get_dummies(df.tweetSource)

description=df.description.fillna('_')
description=[clean(str(i)) for i in description]

y=df.follow

In [13]:
X=description

In [26]:
X=tweetSource

In [27]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split =  train_test_split(X, y, test_size=0.4, random_state =0)

In [15]:
model1=Pipeline([
                #('selector', ItemSelector(key='description')),
                ('tfidf', TfidfVectorizer()),
                ('svd', TruncatedSVD()),
                ('model', RandomForestClassifier(n_jobs=-1)),
            ])

In [16]:
from sklearn.model_selection import GridSearchCV
parameters = {'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
              'tfidf__use_idf': (True, False),
              'svd__n_components': (3,5,10,20,30,50),
              'model__n_estimators': (5,10,50,100),
}
clf = GridSearchCV(model1, parameters, n_jobs=-1)
clf.fit(x_train,y_train)
#http://scikit-learn.org/stable/auto_examples/plot_compare_reduction.html

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ..._jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)], 'tfidf__use_idf': (True, False), 'svd__n_components': (3, 5, 10, 20, 30, 50), 'model__n_estimators': (5, 10, 50, 100)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [17]:
print(clf.best_estimator_)
print(clf.best_score_)
print(clf.best_params_)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
 ..._jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])
0.7181208053691275
{'model__n_estimators': 50, 'svd__n_components': 3, 'tfidf__ngram_range': (1, 3), 'tfidf__use_idf': False}


In [28]:
model2=Pipeline([
                #('selector', ItemSelector(key='tweetSource')),
                ("pca", PCA()), 
                ("univ_select", SelectKBest()),
                ('model', RandomForestClassifier(n_jobs=-1)),
            ])

In [31]:
from sklearn.model_selection import GridSearchCV
parameters = {'pca__n_components': (3,5,7),
              'univ_select__k': (1,3,'all'),
              'model__n_estimators': (5,10,50,100),
}
clf2 = GridSearchCV(model2, parameters, n_jobs=-1)
clf2.fit(x_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('univ_select', SelectKBest(k=10, score_func=<function f_classif at 0x00000215BC6CEB70>)), ('model', RandomForestClassifier(bootstrap=True, class_weight=None, crit..._jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'pca__n_components': (3, 5, 7), 'univ_select__k': (1, 3, 'all'), 'model__n_estimators': (5, 10, 50, 100)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [32]:
print(clf2.best_estimator_)
print(clf2.best_score_)
print(clf2.best_params_)

Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('univ_select', SelectKBest(k=1, score_func=<function f_classif at 0x00000215BC6CEB70>)), ('model', RandomForestClassifier(bootstrap=True, class_weight=None, criterio..._jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])
0.7114093959731543
{'model__n_estimators': 10, 'pca__n_components': 5, 'univ_select__k': 1}
